# Pytorch Linear Regression Example #  
  
이번 노트북에서는 전에 배웠던 requires_grad나 backward를 어떻게 사용할 수 있는지  
Linear Regression 예제를 이용해 알아볼 예정이다.

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

(위의 import statement는 거의 고정으로 나오니 그냥 외우도록 하자.)

In [29]:
num_data = 100
num_epoch = 500

In [30]:
# tensor를 uniform(-10,10)이라는 distribution에서 나온 샘플들로 채워준 뒤 x로 반환.
x = init.uniform_(torch.zeros(num_data,1), -10, 10)
print(x)


tensor([[ 0.9894],
        [-1.4234],
        [-1.3348],
        [ 6.0159],
        [-8.8068],
        [ 4.0964],
        [ 4.9317],
        [ 5.4179],
        [-7.1610],
        [ 1.2331],
        [-3.0109],
        [ 3.4616],
        [ 8.5172],
        [-7.4301],
        [ 5.2134],
        [ 3.6121],
        [ 0.8622],
        [ 4.9028],
        [-9.4950],
        [-8.7607],
        [ 4.7920],
        [-1.7546],
        [ 4.3826],
        [ 4.7746],
        [ 0.1649],
        [ 4.5578],
        [-3.5195],
        [ 4.5933],
        [ 8.7658],
        [ 1.3940],
        [-5.0402],
        [ 2.6794],
        [ 6.4175],
        [-5.5432],
        [-7.4574],
        [-5.5066],
        [ 4.4444],
        [-7.8171],
        [ 1.6071],
        [ 3.0826],
        [-9.8816],
        [ 3.0713],
        [ 6.8746],
        [ 3.7746],
        [-5.8001],
        [ 6.6887],
        [ 2.5512],
        [-5.1637],
        [ 9.0179],
        [ 2.1937],
        [-7.4112],
        [-6.6269],
        [ 6.

In [31]:
# 위와 비슷하게 tensor를 N(0,1) distribution에서 나온 샘플들로 채운 뒤 noise로 반환한다.
noise = init.normal_(torch.zeros(num_data,1), std=1)
print(noise)

tensor([[-0.4075],
        [-2.5302],
        [-0.1124],
        [-0.0932],
        [-1.3691],
        [-0.9295],
        [-0.5958],
        [-0.1905],
        [ 2.2968],
        [-0.3293],
        [-1.4511],
        [ 1.5581],
        [-3.0350],
        [ 2.4790],
        [ 1.5015],
        [ 0.9231],
        [ 0.2583],
        [-0.4152],
        [ 0.8916],
        [ 1.2110],
        [ 1.5201],
        [-0.3739],
        [ 0.4265],
        [ 0.1962],
        [ 0.0585],
        [-0.4154],
        [-0.4969],
        [ 0.7109],
        [-1.0851],
        [ 0.5953],
        [-2.6221],
        [ 1.9986],
        [ 0.1849],
        [-1.8762],
        [ 0.5513],
        [-0.2238],
        [-0.3305],
        [-2.5283],
        [-0.1948],
        [-0.6636],
        [ 0.3920],
        [-0.1208],
        [ 1.8927],
        [ 1.0064],
        [ 0.9968],
        [-0.4645],
        [ 0.3492],
        [-0.2334],
        [-0.5569],
        [-1.3293],
        [ 1.3319],
        [-0.9067],
        [-1.

In [32]:
y = 2*x + 3 # noise가 없는 y값. 우리의 target이 될 것이다.
y_noise = 2*(x+noise) + 3 # noise가 있는 y값. 우리가 실제로 얻은 데이터를 나타낸다.

In [33]:
model = nn.Linear(1,1)

여기서 우리가 만든 model은 Neural Network model이다.  
Neural Network의 다양한 모델 중 Linear 모델을 사용하면 1st order linear function을 모델할 수 있다.  
nn.Linear 모델은 input과 output의 갯수를 지정하게 된다. (우리는 1,1로 설정했으므로 input도 한개, output도 한개이다.)  
Linear 모델은 input과 ouput과의 관계들을 weight로 연결하고, 또한 bias를 가지고 있다.
따라서 원래라면 Y = W*X + B 였을 모델이 우리가 input, output이 1개이므로 y = w*x + b가 된다.  
Linear Regression이란 결국 Residue의 합이 가장 작아지는 w와 b를 구하는 것이므로  
Neural Network 중에 Linear 모델의 **subset**이라 할 수 있다.

In [34]:
loss_func = nn.MSELoss()

이 전 예제에서는 우리가 직접 loss 함수를 수식으로 만들어 주었는데  
사실은 pytorch에서 다양한 loss 함수를 제공한다.  
그 중에서 Lienar Regression에 쓰이는 Mean Square Error를 사용하기로 한다.  
이 함수는 나중에 optimizer의 argument로 제공되는데  
결국 optimizer한테 "이 Loss를 최소화 하는 방향으로 진행해줘." 라고 전달해 주는 것이다.

In [35]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

실제로 학습을 담당하는 객체이다.  
SGD는 Stochastic Gradient Descent의 줄임말로 매 샘플마다 parameter를 업데이트하겠다는 말이다.  
즉, input X가 500개의 x로 이루어져 있으면 매 x마다 업데이트를 한다. (더 자주 업데이트를 한다는 얘기)  
첫번째 arg는 어떤 값들을 업데이트할 것인지에 대한 것인데 우리는 model의 모든 파라미터를 업데이트할 것이다.  
즉, 이 optimizer는 loss func를 model의 모든 파라미터들의 편미분한 값을 가지고 모든 파라미터들을 업데이트하게 될 것이다.  
lr은 learning rate로 얼마나 빠르게 parameter들을 업데이트할 것인가에 대한 값이다.

## Noise가 없는 이상적인 값(y)을 output label 데이터로 활용했을 때 ##

In [36]:

label = y
for i in range(num_epoch):
    optimizer.zero_grad()
    # 새로운 데이터들에 대해 다시 기울기를 구하기 위해 초기화.
    # 기본적으로 zero_grad는 models.parameters() 안의 weight들의 grad를 0으로 초기화 해준다.
    # 그런데 보통 gradient descent technique을 쓰기 위해 쓰이는 batch 연산을 지원하기 위해
    # backward() 함수를 호출할 때마다 계산되는 grad값들은 기본적으로 accumulate한다.
    # 왜냐하면 모았다가 한꺼번에 계산해서 업데이트하는게 더 빠르기 때문이다.
    # weight를 업데이트하고 나서도 grad를 초기화 해주지 않으면 그 전에 사용하던 grad가 그대로 남겨진다.
    # 그래서 완전히 새로운 w를 통해 새로 게산되어야 할 grad가 이전 값에 영향을 받게 되므로
    # 반드시 zero_grad를 호출해 주어야 한다.

    # 차이를 알고 싶다면 밑의 print 구문의 comment를 없애고 실행해보라.
    # 참고: https://stackoverflow.com/questions/62067400/understanding-accumulated-gradients-in-pytorch

    #param = list(model.parameters())
    #print("initial grad = {}".format(param[0].grad))
    output = model(x)

    loss = loss_func(output, label)
    loss.backward()
    #print("grad after backward() = {}".format(param[0].grad))
    optimizer.step()

    if i % 100 == 0:
        print("loss = {}".format(loss.data))

param_list = list(model.parameters())
print(param_list[0].item(), param_list[1].item())

loss = 50.891300201416016
loss = 0.09243965148925781
loss = 0.001758561935275793
loss = 3.345064396853559e-05
loss = 6.363511033669056e-07
2.00000262260437 2.9998888969421387


# 결과 #
- 500번의 반복 이후에는 loss가 거의 0에 수렴하고
- 원래 w=2, b=3으로 수렴하였다.

# Noise가 있는 값(y_noise)을 output label 데이터로 활용했을 때 #


In [37]:
# 위에서 모델 parameter를 다 업데이트 해놨으니 다시 만들어서 초기화
model = nn.Linear(1,1)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# noisy label
label = y_noise
for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(x)

    loss = loss_func(output, label)
    loss.backward()
    #print("grad after backward() = {}".format(param[0].grad))
    optimizer.step()

    if i % 100 == 0:
        print("loss = {}".format(loss.data))

param_list = list(model.parameters())
print(param_list[0].item(), param_list[1].item())

loss = 254.05718994140625
loss = 4.995692253112793
loss = 4.776628494262695
loss = 4.772461414337158
loss = 4.772382736206055
2.0188746452331543 2.898496627807617


# 결과 #
- 당연하게도 Noise가 없는 버전보다는 Loss나 최종 w, b가 차이가 있지만
- 500번의 반복 이후에는 loss 많이 줄어들었고,
- 원래 w=2, b=3으로 수렴하였다.
